In [7]:
import requests

URL = "https://ropsten.infura.io/v3/9aa3d95b3bc440fa88ea12eaa4456161"

payload = {
  "jsonrpc": "2.0",
  "method": "eth_blockNumber",
  "params": [],
  "id": 1,
}
response = requests.post(URL, json=payload)
result = response.json()["result"]
latest_block_number = int(result, 16)
print(f"{latest_block_number:,}")


11,054,972


In [10]:
payload = {
  "jsonrpc": "2.0",
  "method": "eth_getBalance",
  "params": [
    "0x15bFA2af0092ea5A270ac053Af41a101B0F603aB",
    "latest"
  ], 
  "id": 1
}
response = requests.post(URL, json=payload)
print(response.json())
result = response.json()["result"]
current_balance = int(result, 16)
print(f"{current_balance:,}")


{'jsonrpc': '2.0', 'id': 1, 'result': '0x429d069189e0000'}
300,000,000,000,000,000


In [45]:
def wei2eth(wei: int) -> float:
  return round(wei * 10**-18, 16)

def eth2wei(eth: float) -> int:
  return round(eth * 10**18)

def gwei2eth(gwei: int) -> float:
  return gwei * 10**-9

def gwei2wei(gwei: int) -> int:
  return gwei * 10**9

def hex2i(hex: str) -> int:
  return int(hex, 16)

def format_n(n) -> str:
  return f"{n:,}"

def format_hex_wei(hex: str) -> str:
  return format_n(wei2eth(hex2i(hex)))

def payload(method: str, *params):
  return {"jsonrpc": "2.0", "method": method, "params": params, "id": 1}

def rpc(method: str, *params):
  p = payload(method, *params)
  response = requests.post(URL, json=p)
  data = response.json()
  if "error" in data:
    raise Exception(data["error"])
  return data["result"]

result = rpc("eth_getBalance", "0x15bFA2af0092ea5A270ac053Af41a101B0F603aB", "latest")

def eth_get_balance(account: str, block: str = "latest") -> str:
  return rpc("eth_getBalance", account, block)

ACCOUNT1 = "0x15bFA2af0092ea5A270ac053Af41a101B0F603aB"
ACCOUNT2 = "0x0e1176663c2c1B2A19f55FF2b81934cd29937C58"

print(format_hex_wei(eth_get_balance(ACCOUNT1)))
print(format_hex_wei(eth_get_balance(ACCOUNT2)))



0.3
0.3


In [121]:
ACCOUNT2 = "0xc8fCb68700Cd780979d2B71Bc2135c1931F015Ba"

send_money_txn = {
  "from": ACCOUNT1,
  "to": ACCOUNT2,
  "gas": 21_000,
  "gasPrice": hex(gwei2wei(50)),
  "value": hex(eth2wei(0.25)),
  "data": "0x",
}

result = rpc("eth_sendTransaction", send_money_txn)
print(result)

Exception: {'code': -32601, 'message': 'The method eth_sendTransaction does not exist/is not available'}

In [58]:
from eth_account.account import Account

ACCOUNT1_PRIVATE = "0x0316192f8744be9ec18ce67f014ee1e33942cce1bedbfc861d8b8ea298f6caed"

account1 = Account.from_key(ACCOUNT1_PRIVATE)
print(account1.address)

transaction = {
  "type": 2,
  "gas": 23_000,
  "maxFeePerGas": gwei2wei(10),
  "maxPriorityFeePerGas": gwei2wei(10),
  "to": ACCOUNT2,
  "value": hex(eth2wei(0.1)),
  "data": "0x",
  "nonce": 0,
  "chainId": 3,
}

signed_transaction = account1.sign_transaction(transaction)
raw_transaction = signed_transaction.rawTransaction
print(raw_transaction.hex())

result = rpc("eth_sendRawTransaction", raw_transaction.hex())
print(result)

0x15bFA2af0092ea5A270ac053Af41a101B0F603aB
0x02f87403808502540be4008502540be4008259d8940e1176663c2c1b2a19f55ff2b81934cd29937c5888016345785d8a000080c001a00af56be8a7515e21439e6894a0d09cac439006ed450d589d3416748e0c5543dea031df494a428f406b941a35380f3134a8286588a15b5bfc8cfce7a0ed6b289006
0xe6edfd8f8beaace99bfec0dac80dfca4b7540d889dcf701beed47c00ed65ffd4


In [72]:
first_hello_bin = "0x608060405234801561001057600080fd5b50610182806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063ef5fb05b14610030575b600080fd5b61003861004e565b60405161004591906100a7565b60405180910390f35b606060405180606001604052806023815260200161012a60239139905090565b6000610079826100c9565b61008381856100d4565b93506100938185602086016100e5565b61009c81610118565b840191505092915050565b600060208201905081810360008301526100c1818461006e565b905092915050565b600081519050919050565b600082825260208201905092915050565b60005b838110156101035780820151818401526020810190506100e8565b83811115610112576000848401525b50505050565b6000601f19601f830116905091905056fe48656c6c6f2066726f6d2074686520466972737448656c6c6f20636f6e747261637421a2646970667358221220587f9774aed2ad740d3ff29784a9464bbdf322057f9c2873afc99c5949f9de8064736f6c63430008070033"

create_contract_txn = {
  "type": 2,
  "gas": 1_000_000,
  "maxFeePerGas": gwei2wei(21),
  "maxPriorityFeePerGas": gwei2wei(21),
  "data": first_hello_bin,
  "nonce": 2,
  "chainId": 3,
}

signed_create_contract_txn = account1.sign_transaction(create_contract_txn)
raw_cc_txn = signed_create_contract_txn.rawTransaction
print(raw_cc_txn.hex())

result = rpc("eth_sendRawTransaction", raw_cc_txn.hex())
print(result)


0x02f901fd03028504e3b292008504e3b29200830f42408080b901a2608060405234801561001057600080fd5b50610182806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063ef5fb05b14610030575b600080fd5b61003861004e565b60405161004591906100a7565b60405180910390f35b606060405180606001604052806023815260200161012a60239139905090565b6000610079826100c9565b61008381856100d4565b93506100938185602086016100e5565b61009c81610118565b840191505092915050565b600060208201905081810360008301526100c1818461006e565b905092915050565b600081519050919050565b600082825260208201905092915050565b60005b838110156101035780820151818401526020810190506100e8565b83811115610112576000848401525b50505050565b6000601f19601f830116905091905056fe48656c6c6f2066726f6d2074686520466972737448656c6c6f20636f6e747261637421a2646970667358221220587f9774aed2ad740d3ff29784a9464bbdf322057f9c2873afc99c5949f9de8064736f6c63430008070033c080a0250cf53c4b6c8cf8a49880eb0c05ebd70df91c798c9ac693879088a5d2910ee7a037fc094d6a8ddd4b4310671dd856c0ea6608

In [102]:
from Crypto.Hash import keccak

FIRST_HELLO_CONTRACT = "0xc84f554717bf618ab168b93881c3fc1cdab42d49"

say_hello_method = "sayHello()"

hasher = keccak.new(digest_bits=256)
hasher.update(say_hello_method.encode("ascii"))
say_hello_method_id = "0x" + hasher.digest()[:4].hex()

print(say_hello_method_id)

say_hello_call = {
  "to": FIRST_HELLO_CONTRACT,
  "data": say_hello_method_id,
}

result = rpc("eth_call", say_hello_call, "latest")
print(result)


0xef5fb05b
0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002348656c6c6f2066726f6d2074686520466972737448656c6c6f20636f6e7472616374210000000000000000000000000000000000000000000000000000000000


In [120]:
from eth_abi import decode_abi
result_bytes = bytes.fromhex(x[2:])
decode_abi(['string'], result_bytes)


'Hello from the FirstHello contract!'

In [109]:
# call_with_txn = {
#   "type": 2,
#   "gas": 1_000_000,
#   "maxFeePerGas": gwei2wei(21),
#   "maxPriorityFeePerGas": gwei2wei(21),
#   "data": say_hello_method_id,
#   "to": FIRST_HELLO_CONTRACT,
#   "value": "0x",
#   "nonce": 0,
#   "chainId": 3,
# }

# signed_call_with_txn = account1.sign_transaction(call_with_txn)
# raw_call_with_txn = signed_call_with_txn.rawTransaction
# print(raw_call_with_txn.hex())
